# Step 1 - Get and Cleanse Data

In [19]:
#Wikipedia scraping for Toronto postal codes
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_scrape = pd.read_html(url)

In [20]:
#check number of tables found
len(df_scrape)

3

In [21]:
#show table
df_scrape[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [22]:
#create separate dataframe
df_neigh = df_scrape[0]
df_neigh.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
#remove rows with borough Not assigned

# Get names of indexes
indexNames = df_neigh[ df_neigh['Borough'] == 'Not assigned' ].index

# Delete these row indexes from dataFrame
df_neigh.drop(indexNames , inplace=True)
df_neigh.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
df_neigh["Neighbourhood"] = df_neigh.groupby("Postal Code")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
df_neigh = df_neigh.drop_duplicates()

#update index to be Postal Code if it isn't already
#if(df_neigh.index.name != 'Postal Code'):
#    df_neigh = df_neigh.set_index('Postal Code')
    
df_neigh.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [25]:
#if a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough
indexNames = df_neigh['Neighbourhood'] == 'Not assigned'
df_neigh['Neighbourhood'][indexNames] = df_neigh['Borough'][indexNames]
df_neigh.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [26]:
#in the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
df_neigh.shape

(103, 3)

# Step 2 - Add Geospatial Data

In [27]:
#get latitude and longitude data from csv
import pandas as pd
df_coord = pd.read_csv('http://cocl.us/Geospatial_data') 
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
#set the index to be Postal Code
#df_coord.columns = ["Postal Code", "Latitude", "Longitude"]

if(df_coord.index.name != 'Postal Code'):
    df_coord = df_coord.set_index('Postal Code')
if(df_neigh.index.name != 'Postal Code'):
    df_neigh = df_neigh.set_index('Postal Code')

#merge neighbourhood and coordinates dataframes
df_neigh = df_neigh.join(df_coord)

df_neigh.head(11)

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M3B,North York,Don Mills,43.745906,-79.352188
M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [30]:
#reset index
df_neigh.reset_index(inplace=True)
df_neigh

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
